# Lomb-Scargle decomposition Atmosphere time sequences from Merra2

- author Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date 2025-10-27 :
- last update : 2025-02-28 : units
- Kernel @usdf **w_2024_50*
- Office emac : mamba_py311
- Home emac : base (conda)
- laptop : conda_py311

https://docs.astropy.org/en/latest/timeseries/lombscargle.html

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import os

In [ ]:
# where are stored the figures
pathfigs = "figs LombScargleAtmosphFomMerra2"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs)
figtype = ".png"

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from astropy.coordinates.earth import EarthLocation
from datetime import datetime
from pytz import timezone

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy
from scipy.optimize import curve_fit,least_squares

In [ ]:
props = dict(boxstyle='round', facecolor='white', alpha=0.5)

In [ ]:
from astropy.modeling import models

In [ ]:
from numpy.random import lognormal

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time
from astropy.timeseries import TimeSeries

In [ ]:
# Remove to run faster the notebook
#import ipywidgets as widgets
#%matplotlib widget

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
np.__version__

In [ ]:
pd.__version__

In [ ]:
from scipy.fftpack import fft, fftfreq

In [ ]:
from astropy.timeseries import LombScargle

In [ ]:
def convertNumToDatestr(num):
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return pd.to_datetime(datestr)

In [ ]:
def pdf_lognormal(x,a0,mu,sigma):
    """
    """
    pdf = a0*(np.exp(-(np.log(x) - mu)**2 / (2 * sigma**2))/ (x * sigma * np.sqrt(2 * np.pi)))
    return pdf

In [ ]:
FIGXSIZE_1 = 14
FIGYSIZE_1 = 8

FIGXSIZE_0 = 14
FIGYSIZE_0 = 6

In [ ]:
YEAR = 365.25
MONTHS6 = YEAR/2.
MONTHS4 = YEAR/3.
QUARTER = YEAR/4. 
DAY = 1.
MONTH = YEAR/12.
WEEK = 7*DAY

In [ ]:
def fourier_analysis(dates, values, ax, mode = "logxlogy",title="FFT - Spectrum",
                    xlabel="frequency (day)$^{-1}$",ylabel="y-unit $\cdot \sqrt{day}$",label="DFT amplitude", legendout = True, datecut = 59500 ):
    # Centrer les données autour de la moyenne


    if datecut>0:
        index_selected = np.where(dates >= datecut)[0]
        dates = dates[index_selected]
        values = values[index_selected]

    
    values_centered = values - np.mean(values)
    

    # Nombre de points
    N = len(dates)
    # signal duration
    Delta_T = dates.max()-dates.min()

    #frequency spacing
    Delta_f = 1/Delta_T

    # sigma_x
    sigma_x = np.sqrt(np.sum(values_centered**2)/N)
    
    # Intervalle d'échantillonnage (assume 1 jour entre chaque point)
    T = np.mean(np.diff(dates))  # Période d'échantillonnage

    # Fréquence de Nyquist (limite de Shannon)
    f_nyquist = 1 / (2 * T)
    
    # Transformée de Fourier
    fft_values = fft(values_centered)/ np.sqrt(N)
    freqs = fftfreq(N, T)  # Fréquences associées


    # Seulement la moitié du spectre est utile (symétrie)
    positive_freqs = freqs[:N // 2]
    positive_fft_values = np.abs(fft_values[:N // 2])

    # convert in values units x sqrt(day)
    positive_fft_values /= np.sqrt(Delta_f)

    # Tracer le spectre
    #fig, ax = plt.subplots(1,1,figsize=(FIGXSIZE_0, FIGYSIZE_0),layout="constrained")
    
    ax.plot(positive_freqs, positive_fft_values,'ob-' ,ms=5,label=label)

    if mode == "logxliny":
        ax.set_xscale("log")  # Définit l'axe X en échelle logarithmique
        ax.set_yscale("linear")  # Garde l'axe Y en échelle linéaire
    elif mode == "logxlogy":
        ax.set_xscale("log")  # Définit l'axe X en échelle logarithmique
        ax.set_yscale("log")  # Garde l'axe Y en échelle logarithmique
    elif mode == "linxlogy":
        ax.set_xscale("linear")  # Définit l'axe X en  échelle linéaire
        ax.set_yscale("log")  # Garde l'axe Y en échelle logarithmique
    elif mode == "linxliny":
        ax.set_xscale("linear")  # Définit l'axe X en échelle linéaire
        ax.set_yscale("linear")  # Garde l'axe Y en échelle linéaire
        
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    

    ax.axvline(1/YEAR, color='r', linestyle='-', label="Cycle : 365 days - 1 year")
    ax.axvline(1/MONTHS6, color='r', linestyle='--', label="Cycle : 182.6 days - 6 months")
    ax.axvline(1/MONTHS4, color='r', linestyle=':', label="Cycle : 121.7 days - 4 months")
    ax.axvline(1/QUARTER, color='r', linestyle='-.', label="Cycle : 91.3 days - 3 months")
    ax.axvline(1/MONTH, color='r', linestyle=':', label="Cycle : 30.4 days - 1 month")
    ax.axvline(1/WEEK, color='purple', linestyle='--', label="Cycle : 7 days - 1 week")
    ax.axvline(DAY, color='purple', linestyle='-', label="Cycle : 1 day ")
    ax.axvline(1./(0.5*DAY), color='purple', linestyle='-.', label="Cycle : 0.5 day ")

    ax.axvline(f_nyquist, color='g', linestyle='--', label=f"Nyquist frequency({f_nyquist:.3f} cycles/days)")

    txtstr_sigma = "$\sigma_x$ = " + f" {sigma_x:0.3f}" 
    txtstr_Ts = "$T_s$ = "  + f" {T:0.3f} days" 
    txtstr_Fs = "$f_s$ = "  + f" {Delta_f:0.3f} / days" 

    txtstr_info = "\n".join([txtstr_sigma,txtstr_Ts ,txtstr_Fs])
    ax.text(0.01, 0.95, txtstr_info, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)
    

    if legendout:
        ax.legend(bbox_to_anchor=(1.05, 1.05),fontsize=12)
    else:
        ax.legend(fontsize=10,fancybox=True, framealpha=0.5)
        


In [ ]:
def LombScargle_analysis(dates, values, ax ,mode = "logxlogy",title="LombScargle - Spectrum",
                    xlabel="frequency (day)$^{-1}$",ylabel="y-unit",label="Lomb Scargle", legendout = True, datecut = 0 ):
    # Centrer les données autour de la moyenne


    if datecut>0:
        index_selected = np.where(dates >= datecut)[0]
        dates = dates[index_selected]
        values = values[index_selected]

    
    values_centered = values - np.mean(values)
    

    # Nombre de points
    N = len(dates)

    # sigma
    sigma_x = np.sqrt(np.sum(values_centered**2)/N)
    
    # Intervalle d'échantillonnage (assume 1 jour entre chaque point)
    T = np.mean(np.diff(dates))  # Période d'échantillonnage

    # Fréquence de Nyquist (limite de Shannon)
    f_nyquist = 1 / (2 * T)
    

    freqs, power = LombScargle(dates, values_centered).autopower()
   
    ax.plot(freqs, np.sqrt(power),'ob-' ,ms=5,label=label)

    if mode == "logxliny":
        ax.set_xscale("log")  # Définit l'axe X en échelle logarithmique
        ax.set_yscale("linear")  # Garde l'axe Y en échelle linéaire
    elif mode == "logxlogy":
        ax.set_xscale("log")  # Définit l'axe X en échelle logarithmique
        ax.set_yscale("log")  # Garde l'axe Y en échelle logarithmique
    elif mode == "linxlogy":
        ax.set_xscale("linear")  # Définit l'axe X en  échelle linéaire
        ax.set_yscale("log")  # Garde l'axe Y en échelle logarithmique
    elif mode == "linxliny":
        ax.set_xscale("linear")  # Définit l'axe X en échelle linéaire
        ax.set_yscale("linear")  # Garde l'axe Y en échelle linéaire
        
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    

    ax.axvline(1/YEAR, color='r', linestyle='-', label="Cycle : 365 days - 1 year")
    ax.axvline(1/MONTHS6, color='r', linestyle='--', label="Cycle : 182.6 days - 6 months")
    ax.axvline(1/MONTHS4, color='r', linestyle=':', label="Cycle : 121.7 days - 4 months")
    ax.axvline(1/QUARTER, color='r', linestyle='-.', label="Cycle : 91.3 days - 3 months")
    ax.axvline(1/MONTH, color='r', linestyle=':', label="Cycle : 30.4 days - 1 month")
    ax.axvline(1/WEEK, color='purple', linestyle='--', label="Cycle : 7 days - 1 week")
    ax.axvline(DAY, color='purple', linestyle='-', label="Cycle : 1 day ")
    ax.axvline(1./(0.5*DAY), color='purple', linestyle='-.', label="Cycle : 0.5 day ")

    ax.axvline(f_nyquist, color='g', linestyle='--', label=f"Nyquist frequency({f_nyquist:.3f} cycles/days)")

    txtstr_sigma = "$\sigma_x$ = " + f" {sigma_x:0.3f}" 
    ax.text(0.01, 0.95, txtstr_sigma, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)
    

    if legendout:
        ax.legend(bbox_to_anchor=(1.05, 1.05),fontsize=12)
    else:
        ax.legend(fontsize=10,fancybox=True, framealpha=0.5)
        


## Configuration

In [ ]:
observing_location = EarthLocation.of_site('Rubin Observatory')
tz = timezone('America/Santiago')

### MERRA2

In [ ]:
filename_m2 = "../../SpectroMerra2/MerradataMerged/Merge_inst1_2d_asm_Nx_M2I1NXASM-2021-2024.csv"
filename_m2b = "../../SpectroMerra2/MerradataMerged/Merge_tavg1_2d_aer_Nx_M2T1NXAER-2021-2024.csv"

In [ ]:
df_m = pd.read_csv(filename_m2)
df_mb = pd.read_csv(filename_m2b)

In [ ]:
Nm = len(df_m)
Nmb = len(df_mb)
print("Number of points :: ",Nm,Nmb)

In [ ]:
df_mb.columns

In [ ]:
TMIN = pd.to_datetime(df_m.time.min())
TMAX = pd.to_datetime(df_m.time.max())

### Convert in MJD

In [ ]:
df_m["mjd"] = Time(pd.to_datetime(df_m.time.values)).mjd
df_mb["mjd"] = Time(pd.to_datetime(df_mb.time.values)).mjd

In [ ]:
mjd_zoom_start = Time("2024-01-01").mjd
mjd_zoom_stop = Time("2025-06-30").mjd

In [ ]:
mjd_obs_start = df_m["mjd"].min() 
mjd_obs_stop = df_m["mjd"].max() 

## PWV

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m")

fig = plt.figure(figsize=(18,10))
gs = GridSpec(2, 1,figure=fig)
#gs = GridSpec(1, 1,figure=fig)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
        
leg1=ax1.get_legend()
leg2=ax2.get_legend()


ax1.plot(pd.to_datetime(df_m.Time.values), df_m.TQV.values,c="b",lw=0.5,label="Merra2")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax1.set_title("Precipitable water vapor from Merra2")
ax1.legend()
ax1.set_ylabel("PWV (mm)")
#ax.set_xlim(TMIN,TMAX)

data = df_m.TQV.values
mean = np.mean(data)
median = np.median(data)
std = np.std(data)
textstr = "\n".join((f"Expected max-range for PWV : ",
                     f"- average : {mean:.2f} mm",
                     f"- median : {median:.2f} mm",
                     f"- sigma : {std:.2f} mm",     
                    ))
ax1.text(0.05, 0.95, textstr, transform=ax1.transAxes, fontsize=14,verticalalignment='top', bbox=props)


ax2.plot(df_m.mjd, df_m.TQV.values,c="b",lw=0.5,label="Merra2")
ax2.set_xlabel("time (MJD)")
ax2.legend()
ax2.set_ylabel("PWV (mm)")

figname =f"{pathfigs}/pwv_allpoints_merra2"+figtype
fig.savefig(figname)
plt.show()


### FFT for  PWV

In [ ]:
dates = df_m.mjd
values = df_m.TQV.values
figname =f"{pathfigs}/pwv_FFT_merra2"+figtype

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
fourier_analysis(dates, values, ax=ax ,mode= "logxliny",
                 title = "FFT : PWV (Absolute value of FFT)",
                 #xlabel="frequency (days$^{-1}$)",
                 ylabel="mm $\cdot \sqrt{day}$",
                 label="Merra2 PWV")
fig.savefig(figname)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
LombScargle_analysis(dates, values, ax=ax ,mode= "logxliny",
                 title = "LombScargle : PWV  ($\sqrt{LombScargle}$)",
                 #xlabel="frequency (days$^{-1}$)",
                 ylabel="mm",
                 label="Merra2 PWV amplitude")
figname =f"{pathfigs}/pwv_LombScargle_merra2"+figtype
fig.savefig(figname)
plt.show()

In [ ]:
fig,axs = plt.subplots(1,2,figsize = (FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
ax1,ax2 = axs.flatten()

fourier_analysis(dates, values, ax=ax1 ,mode= "logxliny",
                 title = "FFT : PWV (Absolute value of FFT)",
                 #xlabel="frequency (days$^{-1}$)",
                 ylabel="mm $\cdot \sqrt{day}$",
                 label="Merra2 PWV amplitude",legendout = False)

LombScargle_analysis(dates, values, ax=ax2 ,mode= "logxliny",
                 title = "LombScargle : PWV ($\sqrt{LombScargle}$)",
                 #xlabel="frequency (days$^{-1}$)",
                 ylabel="mm",
                 label="Merra2 PWV amplitude",legendout = False)

figname =f"{pathfigs}/pwv_FFTLombScargle_merra2"+figtype
fig.savefig(figname)
plt.show()

## Ozone

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m")

fig = plt.figure(figsize=(18,10))
gs = GridSpec(2, 1,figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
        
leg1=ax1.get_legend()
leg2=ax2.get_legend()


ax1.plot(pd.to_datetime(df_m.Time.values), df_m.TO3.values,c="r",lw=0.5,label="Merra2")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax1.set_title("Ozone from Merra2")
#ax1.legend()
ax1.set_ylabel("Ozone (DU)")
#ax.set_xlim(TMIN,TMAX)

data = df_m.TO3.values
mean = np.mean(data)
median = np.median(data)
std = np.std(data)
textstr = "\n".join((f"Expected range for Ozone : ",
                     f"- average : {mean:.2f} DU",
                     f"- median : {median:.2f} DU",
                     f"- sigma : {std:.2f} DU",     
                    ))
ax1.text(0.05, 0.95, textstr, transform=ax1.transAxes, fontsize=14,verticalalignment='top', bbox=props)

ax2.plot(df_m.mjd, df_m.TO3.values,c="r",lw=0.5,label="Merra2")
ax2.set_xlabel("time (MJD)")
ax2.legend()
ax2.set_ylabel("Ozone (DU)")


figname =f"{pathfigs}/ozone_allpoints_merra2"+figtype
fig.savefig(figname)
plt.show()


### FFT for Ozone

In [ ]:
dates = df_m.mjd
values = df_m.TO3.values
figname =f"{pathfigs}/ozone_FFT_merra2"+figtype

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
fourier_analysis(dates, values, ax=ax ,mode= "logxliny",
                 title = "FFT : Ozone (Absolute value of FFT)",
                 #xlabel="frequency (days$^{-1}$)",
                 ylabel="$DU \cdot \sqrt{day}$",
                 label="Merra2 Ozone")
fig.savefig(figname)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
LombScargle_analysis(dates, values, ax=ax ,mode= "logxliny",
                 title = "LombScargle : Ozone ($\sqrt{LombScargle}$)",
                 #xlabel="frequency (days$^{-1}$)",
                 ylabel="DU",
                 label="Merra2 Ozone")
figname =f"{pathfigs}/ozone_LombScargle_merra2"+figtype
fig.savefig(figname)
plt.show()

In [ ]:
fig,axs = plt.subplots(1,2,figsize = (FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
ax1,ax2 = axs.flatten()

fourier_analysis(dates, values, ax=ax1 ,mode= "logxliny",
                 title = "FFT : Ozone (Absolute value of FFT)",
                 #xlabel="frequency (days$^{-1}$)",
                 ylabel="$DU \cdot \sqrt{day}$",
                 label="Merra2 Ozone",legendout = False)

LombScargle_analysis(dates, values, ax=ax2 ,mode= "logxliny",
                 title = "LombScargle : Ozone (($\sqrt{LombScargle}$)",
                 #xlabel="frequency (days$^{-1}$)",
                 ylabel="DU",
                 label="Merra2 Ozone",legendout = False)

figname =f"{pathfigs}/ozone_FFTLombScargle_merra2"+figtype
fig.savefig(figname)
plt.show()

## Aerosol VAOD

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m")

fig = plt.figure(figsize=(18,10))
gs = GridSpec(2, 1,figure=fig)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
        
leg1=ax1.get_legend()
leg2=ax2.get_legend()


ax1.plot(pd.to_datetime(df_mb.Time.values), df_mb.TOTEXTTAU.values,c="g",lw=0.5,label="Merra2")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax1.set_title("VAOD from Merra2")
ax1.legend()
ax1.set_ylabel("VAOD")
#ax.set_xlim(TMIN,TMAX)
data = df_mb.TOTEXTTAU.values
mean = np.mean(data)
median = np.median(data)
std = np.std(data)
textstr = "\n".join((f"Expected max-range for VAOD : ",
                     f"- average : {mean:.3f}",
                     f"- median : {median:.3f}",
                     f"- sigma : {std:.3f}",     
                    ))
ax1.text(0.05, 0.95, textstr, transform=ax1.transAxes, fontsize=14,verticalalignment='top', bbox=props)

ax2.plot(df_mb.mjd, df_mb.TOTEXTTAU.values,c="g",lw=0.5,label="Merra2")
ax2.set_xlabel("time (MJD)")
ax2.legend()
ax2.set_ylabel("VAOD")


figname =f"{pathfigs}/vaod_allpoints_merra2"+figtype
fig.savefig(figname)
plt.show()


### FFT for aerosols - VAOD

In [ ]:
dates = df_mb.mjd
values = df_mb.TOTEXTTAU.values
figname =f"{pathfigs}/vaod_FFT_merra2"+figtype

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
fourier_analysis(dates, values, ax=ax ,mode= "logxliny",
                 title = "FFT : VAOD (Absolute value of FFT)",
                 #xlabel="frequency (days$^{-1}$)",
                 #ylabel="VAOD",
                 label="Merra2 VAOD")
fig.savefig(figname)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
LombScargle_analysis(dates, values, ax=ax ,mode= "logxliny",
                 title = "LombScargle : VAOD ($\sqrt{LombScargle}$)",
                 #xlabel="frequency (days$^{-1}$)",
                 #ylabel="",
                 label="Merra2 VAOD")
figname =f"{pathfigs}/vaod_LombScargle_merra2"+figtype
fig.savefig(figname)
plt.show()

In [ ]:
fig,axs = plt.subplots(1,2,figsize = (FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
ax1,ax2 = axs.flatten()

fourier_analysis(dates, values, ax=ax1 ,mode= "logxliny",
                 title = "FFT : VAOD (Absolute value of FFT)",
                 #xlabel="frequency (days$^{-1}$)",
                 #ylabel="VAOD",
                 label="Merra2 VAOD",legendout = False)

LombScargle_analysis(dates, values, ax=ax2 ,mode= "logxliny",
                 title = "LombScargle : VAOD ($\sqrt{LombScargle}$)",
                 #xlabel="frequency (days$^{-1}$)",
                 #ylabel="",
                 label="Merra2 VAOD",legendout = False)

figname =f"{pathfigs}/vaod_FFTLombScargle_merra2"+figtype
fig.savefig(figname)
plt.show()


## Aerosol Angstrom

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m")

fig = plt.figure(figsize=(18,10))
gs = GridSpec(2, 1,figure=fig)

ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
        
leg1=ax1.get_legend()
leg2=ax2.get_legend()


ax1.plot(pd.to_datetime(df_mb.Time.values), df_mb.TOTANGSTR.values,c="purple",lw=0.5,label="Merra2")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax1.set_title("Angstrom from Merra2")
ax1.legend()
ax1.set_ylabel("Angstrom")
#ax.set_xlim(TMIN,TMAX)
data = df_mb.TOTANGSTR.values
mean = np.mean(data)
median = np.median(data)
std = np.std(data)
textstr = "\n".join((f"Expected max-range for Angstrom : ",
                     f"- average : {mean:.3f}",
                     f"- median : {median:.3f}",
                     f"- sigma : {std:.3f}",     
                    ))
ax1.text(0.05, 0.95, textstr, transform=ax1.transAxes, fontsize=14,verticalalignment='top', bbox=props)


ax2.plot(df_mb.mjd, df_mb.TOTANGSTR.values,c="purple",lw=0.5,label="Merra2")
ax2.set_xlabel("time (MJD)")
ax2.legend()
ax2.set_ylabel("Angstrom")



figname =f"{pathfigs}/vaodangstrom_allpoints_merra2"+figtype
fig.savefig(figname)
plt.show()

### FFT For Aerosol Angstrom

In [ ]:
dates = df_mb.mjd
values = df_mb.TOTANGSTR.values
figname =f"{pathfigs}/angstrom_FFT_merra2"+figtype

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
fourier_analysis(dates, values, ax=ax ,mode= "logxliny",
                 title = "FFT : Angstrom Exponent (Absolute value of FFT)",
                 #xlabel="frequency (days$^{-1}$)",
                 #ylabel="Angstrom exp",
                 label="Merra2 Angstrom exponent")
fig.savefig(figname)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize = (FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
LombScargle_analysis(dates, values, ax=ax ,mode= "logxliny",
                 title = "LombScargle : Angstrom exponent ($\sqrt{LombScargle}$)",
                 #xlabel="frequency (days$^{-1}$)",
                 #ylabel="",
                 label="Merra2 Angstrom exponent")
figname =f"{pathfigs}/angstrom_LombScargle_merra2"+figtype
fig.savefig(figname)
plt.show()

In [ ]:
fig,axs = plt.subplots(1,2,figsize = (FIGXSIZE_0,FIGYSIZE_0),layout="constrained")
ax1,ax2 = axs.flatten()

fourier_analysis(dates, values, ax=ax1 ,mode= "logxliny",
                 title = "FFT : Angstrom Exponent (Absolute value of FFT)",
                 #xlabel="frequency (days$^{-1}$)",
                 #ylabel="Angstrom exp",
                 label="Merra2 Angstrom amplitude",legendout = False)

LombScargle_analysis(dates, values, ax=ax2 ,mode= "logxliny",
                 title = "LombScargle : Angstrom exponent ($\sqrt{LombScargle}$)",
                 #xlabel="frequency (days$^{-1}$)",
                 #ylabel="",
                 label="Merra2 Angstrom exponent",legendout = False)

figname =f"{pathfigs}/angstrom_FFTLombScargle_merra2"+figtype
fig.savefig(figname)
plt.show()
